In [360]:
import os
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
import plotly.express as px
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import plotly.graph_objects as go
from jupyterlab_dash import AppViewer
import numpy as np

#### File exported from Kobo

In [361]:
filepath = "./data/Verkehrssicherheit beim Fahrradfahren - latest version - labels - 2020-05-19-10-28-30.csv"
plot_dir = "."
mapbox_access_token = "pk.eyJ1IjoiY2hsdWR3aWciLCJhIjoiY2pxM3p6cHp4MWpvaTN4cHBhOTA1aWZ4diJ9.FS-wzKCyyUVqHIADwgTkEw"

#### Read data and reformat

In [392]:
data = pd.read_csv(filepath, sep=";").iloc[2:].set_index("_index")

In [393]:
color_dict = {"sehr gut": 'rgb(0,104,55)', "gut": 'rgb(102,189,99)', "tendenziell eher gut": 'rgb(217,239,139)',
              "tendenziell eher schlecht": 'rgb(254,224,139)', "schlecht":'rgb(244,109,67)', "sehr schlecht": 'rgb(165,0,38)'}
    

remove trailing spaces

In [394]:
data = data.applymap(lambda x: x.rstrip() if isinstance(x, str) else x)

Convert lat lon to float

In [395]:
data = data.rename(columns={"_Nehme deinen Standort auf_longitude": "lon", "_Nehme deinen Standort auf_latitude": "lat"})
data["lat"] = data["lat"].astype("float")
data["lon"] = data["lon"].astype("float")

Remove unnecessary columns

In [396]:
drop_cols = ['subscriberid', 'deviceid', '_uuid', '_submission_time', '_validation_status','_id', 'start', 'end',
            '_Nehme deinen Standort auf_altitude', '_Nehme deinen Standort auf_precision', 'Nehme deinen Standort auf']
data.drop(drop_cols, axis=1, inplace=True)

Replace Nan

In [397]:
data = data.fillna("-")

In [398]:
#data.columns

Add description for pop ups

In [399]:
data = pd.concat([data]*3, ignore_index=True)
data['Wie gut findest du den Radweg?'] = color_dict.keys()
data["lon"] = data["lon"] + np.random.randn(6)*0.005
data["lat"] = data["lat"] + np.random.randn(6)*0.005

In [400]:
#print(px.colors.diverging.RdYlGn)

## Create plots

In [401]:
def make_plot(criteria, sub_criteria, data, outfile, show=False):
    
    color_dict = {"sehr gut": 'rgb(0,104,55)', "gut": 'rgb(102,189,99)', "tendenziell eher gut": 'rgb(217,239,139)',
              "tendenziell eher schlecht": 'rgb(254,224,139)', "schlecht":'rgb(244,109,67)', "sehr schlecht": 'rgb(165,0,38)'}
    
    sub_data = data.loc[:, sub_criteria]
    sub_data["desc"] = sub_data.apply(lambda row: "<br>".join(["<b>{}</b>: <i>{}</i>".format(k, v) 
                                                               for k, v in zip(row.index, row) 
                                                               if k not in ["lat", "lon", criteria]]), axis=1)
    fig = go.Figure()
    for key, col in color_dict.items():
        key_data = sub_data.loc[sub_data[criteria] == key]
        fig.add_trace(go.Scattermapbox(
            lat=key_data["lat"],
            lon=key_data["lon"],
            mode='markers',
            marker=go.scattermapbox.Marker(
                size=10, 
                color=col,
            ),
            text=key_data["desc"],
            hoverinfo="text",
            name=key))    

    fig.update_layout(
        hovermode='closest',
        height=800, 
        title=criteria,
        showlegend=True,
        legend_title_text='Bewertung',
        mapbox=dict(
            accesstoken=mapbox_access_token,
            bearing=0,
            center=go.layout.mapbox.Center(
                lat=49.408404,
                lon=8.692645
            ),
            zoom=12
        )
    )
    if show:
        fig.show()
    fig.write_html(outfile)

## Plot 1

In [402]:
criteria = "Wie gut findest du den Radweg?"
sub_criteria = [criteria, 'lat', 'lon', 'Wie gut findest du die technische Ausstattung der Radinfrastruktur?',
               'Wie ist dein generelles Sicherheitsgefühl hier?', 'Wie ist dein generelles Fahrgefühl hier?']
outfilename = os.path.join(plot_dir, "plot1.html")
make_plot(criteria, sub_criteria, data, outfilename)

## Plot 2

In [403]:
criteria = "Wie gut findest du die technische Ausstattung der Radinfrastruktur?"
sub_criteria= [criteria, 'lat', 'lon', 'Ausstattung: ', 'Bauliche oder markierte Trennung zu FußgängerInnen',
       'Bauliche oder markierte Trennung von Autos', 'Bordsteinabsenkungen',
       'Beleuchtung', 'Radwegbelag', 'Radwegmarkierung', 
       'Rote Markierung an Kreuzungen','gibt es eine zusätzliche technische Ausstattung?'] # 'Radwegbreite ',
outfilename = os.path.join(plot_dir, "plot2.html")
make_plot(criteria, sub_criteria, data, outfilename)

## Plot 3

In [404]:
criteria = "Wie ist dein generelles Sicherheitsgefühl hier?"
sub_criteria= [criteria, 'lat', 'lon', 'Gefahren:',
       'Sichtbarkeit für AutofahrerInnen', 'Abstand zu Autos',
       'Abstand zu FußgängerInnen', 'Angemessene Geschwindigkeit der Autos',
       'Gibt es eine zusätzliche Gefahrenquelle? ']
outfilename = os.path.join(plot_dir, "plot3.html")
make_plot(criteria, sub_criteria, data, outfilename)

## Plot 4

In [405]:
criteria = "Wie ist dein generelles Fahrgefühl hier?"
sub_criteria = [criteria, 'lat', 'lon', 'Einflüsse:',
       'Fahrfluss (z.B. grüner Pfeil für Radfahrende, Fahrradampel, ...)',
       'Luftqualität', 'Lärmbelastung', 'Begrünung', 'Übersichtlichkeit',
       'Beschattung', 'Gibt es eine zusätzliche Gefahrenquelle? .1']
outfilename = os.path.join(plot_dir, "plot4.html")
make_plot(criteria, sub_criteria, data, outfilename)

In [ ]:
# Build AppViewer 
viewer = AppViewer()

In [ ]:
criteria = [{"label": x, "value": x} for x in data.columns]

In [ ]:
# Build App
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
mapbox_access_token = "pk.eyJ1IjoiY2hsdWR3aWciLCJhIjoiY2pxM3p6cHp4MWpvaTN4cHBhOTA1aWZ4diJ9.FS-wzKCyyUVqHIADwgTkEw"
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div([
    html.Div([
        html.Label("Kriterien"),
        dcc.Dropdown(id="criteria", multi=False, value="Wie gut findest du den Radweg?", options=criteria),
    ], style={'width': '70%', "display":"inline-block", 'margin': '5px 5px 5px 5px'}),
    html.Div([
        dcc.Graph(id='graph')
    ], style={'width': '100%', 'height': '1000px'})
])

@app.callback(
    Output('graph', 'figure'),
    [Input('criteria', 'value')]
)
def set_map(criteria):
    #px.set_mapbox_access_token("pk.eyJ1IjoiY2hsdWR3aWciLCJhIjoiY2pxM3p6cHp4MWpvaTN4cHBhOTA1aWZ4diJ9.FS-wzKCyyUVqHIADwgTkEw")
    #fig = px.scatter_mapbox(data, lat="lat", lon="lon", color=criteria, hover_data=[criteria], range_color=[0,5],hoverinfo="text",
    #                  color_continuous_scale=px.colors.diverging.RdYlGn, size_max=15, zoom=12, height=750)
    #fig.show()
    
    fig = go.Figure(go.Scattermapbox(
            lat=data["lat"],
            lon=data["lon"],
            mode='markers',
            marker_color=data[criteria],
            marker=go.scattermapbox.Marker(
                size=10, 
                showscale=True,
                cmax=5,
                cmin=0,
                colorscale=px.colors.diverging.RdYlGn
            ),
            text=data["desc"],
            hoverinfo="text",
        ))

    fig.update_layout(
        hovermode='closest',
        height=800, 
        mapbox=dict(
            accesstoken=mapbox_access_token,
            bearing=0,
            center=go.layout.mapbox.Center(
                lat=49.408404,
                lon=8.692645
            ),
            zoom=12
        )
    )

    fig.show()
    
    return fig
    
viewer.show(app)